In [7]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from evraz_anime_analytics.helper import load_data, preprocess_data, predict_date
from evraz_anime_analytics.predict import predict

In [2]:
from datetime import datetime


last_repairment_dates = [
    datetime(2023, 2, 9),
    datetime(2023, 1, 19),
    datetime(2023, 2, 2),
    datetime(2023, 2, 13),
    datetime(2023, 1, 25),
    datetime(2023, 2, 6),
]

WARNING_PENALTY_LEN_COEF = 0.05
WARNING_PENALTY_DIFF_COEF = 0.25
ALARM_PENALTY_LEN_COEF = 0.3
ALARM_PENALTY_DIFF_COEF = 2.275

FREQ = "15min"
MAX_REPAIR_DURATION = 20

In [3]:
ordf = load_data("../sample_data.pqt")
ordf = preprocess_data(ordf)
ordf

/Users/sostema/dev/evraz-hack-misis-anime/analytics/src/evraz_anime_analytics/preprocessing.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, df_groupby.columns[column_id]] = df_groupby[df_groupby.columns[column_id]]


,exhauster_id,bearing_1.temperature.value,bearing_1.temperature.alarm_max,bearing_1.temperature.alarm_min,bearing_1.temperature.warning_max,bearing_1.temperature.warning_min,bearing_1.vibration.vibration_axial.value,bearing_1.vibration.vibration_axial.alarm_max,bearing_1.vibration.vibration_axial.alarm_min,bearing_1.vibration.vibration_axial.warning_max,...,gate_valve.gas_valve_closed,gate_valve.gas_valve_open,gate_valve.gas_valve_position,drive.rotor_current,drive.rotor_voltage,drive.stator_current,drive.stator_voltage,oil.oil_level,oil.oil_pressure,work.is_working
created_at,,,,,,,,,,,,,,,,,,,,,
2023-01-25 06:32:25.125007,0,62.800003,75.0,0.0,65.0,0.0,0.381944,7.1,0.0,4.5,...,False,True,NaN,NaN,NaN,180.850937,NaN,83.547989,2.156743,True
2023-01-25 06:32:25.125007,1,54.299995,75.0,0.0,65.0,0.0,2.552083,7.1,0.0,4.5,...,False,True,NaN,NaN,NaN,167.533020,NaN,73.552475,2.091372,True
2023-01-25 06:32:25.125007,2,47.899994,75.0,0.0,65.0,0.0,0.964621,7.1,0.0,4.5,...,False,True,10.0,1978.0,-3.0,1823.000000,106.0,72.699997,0.822000,True
2023-01-25 06:32:25.125007,3,52.400002,75.0,0.0,65.0,0.0,0.410880,7.1,0.0,4.5,...,False,True,10.0,1693.0,2.0,1897.000000,106.0,74.800003,0.907000,True
2023-01-25 06:32:25.125007,5,19.599998,75.0,0.0,65.0,0.0,0.023148,7.1,0.0,4.5,...,True,True,10.0,73.0,0.0,-1.000000,0.0,86.900002,-0.058000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-18 07:48:25.439816,0,62.500000,75.0,0.0,65.0,0.0,0.254630,7.1,0.0,4.5,...,False,True,NaN,NaN,NaN,189.299408,NaN,79.124199,2.135258,True
2023-02-18 07:48:25.439816,1,53.500000,75.0,0.0,65.0,0.0,2.343750,7.1,0.0,4.5,...,False,True,NaN,NaN,NaN,173.741562,NaN,90.318069,2.093143,True
2023-02-18 07:48:25.439816,2,48.699997,75.0,0.0,65.0,0.0,1.011360,7.1,0.0,4.5,...,False,True,10.0,1650.0,-1.0,2306.000000,106.0,68.900002,0.803000,True


In [4]:
def draw_plot(exdf_sample):
    fig = go.Figure()
    regions = get_regions(exdf_sample)
    for region in regions:
        fig.add_vrect(
            x0=exdf_sample.index[region[0]],
            x1=exdf_sample.index[region[-1]],
            fillcolor="LightSalmon",
            opacity=0.15,
        )
    for bearing_id in range(1, 10):
        for k in [
            "temperature",
            "vibration.vibration_axial",
            "vibration.vibration_horizontal",
            "vibration.vibration_vertical",
        ]:
            sel_col = f"bearing_{bearing_id}.{k}.value"
            if sel_col in df.columns:
                # fig.add_trace(
                #     go.Line(
                #         x=exdf_sample.index,
                #         y=exdf_sample[sel_col],
                #         name=sel_col,
                #     )
                # )
                out_of_norm_points = exdf_sample[
                    exdf_sample[sel_col]
                    > exdf_sample[f"bearing_{bearing_id}.{k}.warning_max"]
                ]
                fig.add_trace(
                    go.Scatter(
                        x=out_of_norm_points.index,
                        y=out_of_norm_points[sel_col],
                        name=f"{sel_col} out of norm",
                    )
                )
    # fig.add_trace(
    #     go.Line(
    #         x=exdf_sample.index,
    #         y=exdf_sample["gate_valve.gas_valve_position"],
    #         name="gate_valve.gas_valve_position",
    #     )
    # )
    # fig.add_trace(
    #     go.Line(
    #         x=exdf_sample.index,
    #         y=exdf_sample["oil.oil_level"],
    #         name="oil.oil_level",
    #     )
    # )
    # fig.add_trace(
    #     go.Line(
    #         x=exdf_sample.index,
    #         y=exdf_sample["oil.oil_pressure"],
    #         name="oil.oil_pressure",
    #     )
    # )
    # fig.add_trace(
    #     go.Line(
    #         x=exdf_sample.index,
    #         y=exdf_sample["gas_collector.temperature_before"],
    #         name="gas_collector.temperature_before",
    #     )
    # )
    # fig.add_trace(
    #     go.Line(
    #         x=exdf_sample.index,
    #         y=exdf_sample["cooler_oil.temperature_after"],
    #         name="cooler_oil.temperature_after",
    #     )
    # )
    # fig.add_trace(
    #     go.Line(
    #         x=exdf_sample.index,
    #         y=exdf_sample["cooler_oil.temperature_before"],
    #         name="cooler_oil.temperature_before",
    #     )
    # )
    # fig.add_trace(
    #     go.Line(
    #         x=exdf_sample.index,
    #         y=exdf_sample["drive.stator_current"],
    #         name="drive.stator_current",
    #     )
    # )
    fig.show()

In [8]:
predict(ordf, last_repairment_dates)

[(0, datetime.date(2023, 3, 1)),
 (1, datetime.date(2023, 2, 20)),
 (2, datetime.date(2023, 2, 22)),
 (3, datetime.date(2023, 3, 6)),
 (4, datetime.date(2023, 2, 20)),
 (5, datetime.date(2023, 2, 27))]